In [5]:
import os
import cv2
from PIL import Image
import pickle
import sys
import dlib
import numpy as np
from skimage import io
from math import sqrt
from csv import writer, reader
from sklearn.preprocessing import StandardScaler
import pandas as pd

# To compute the distances of all the image in ..image folder

In [7]:
#------------------------------------------------------------------------------------------------------
def faceLandmarks(file_name, label):
    predictor_model = "shape_predictor_68_face_landmarks.dat"

    # Create a HOG face detector using the built-in dlib class
    face_detector = dlib.get_frontal_face_detector()
    face_pose_predictor = dlib.shape_predictor(predictor_model)

    #win = dlib.image_window()

    # Take the image file name from the command line
    #file_name = r"C:\Users\amrsa\Downloads\123.jpg"

    # Load the image
    image = io.imread(file_name)
    

    # Run the HOG face detector on the image data
    detected_faces = face_detector(image, 1)

    print("Found {} faces in the image file {}".format(len(detected_faces), file_name))

    # Show the desktop window with the image
    #win.set_image(image)



    # Loop through each face we found in the image
    for i, face_rect in enumerate(detected_faces):

        # Detected faces are returned as an object with the coordinates 
        # of the top, left, right and bottom edges
        print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))

        # Draw a box around each face we found
        #win.add_overlay(face_rect)
        
        # Get the the face's pose
        pose_landmarks = face_pose_predictor(image, face_rect)
        
        # Draw the face landmarks on the screen.
        #win.add_overlay(pose_landmarks)

        #getting 68 points
        vec = np.empty([68, 2], dtype = int)
        for b in range(68):
            vec[b][0] = pose_landmarks.part(b).x
            vec[b][1] = pose_landmarks.part(b).y
            #print(pose_landmarks.part(b).x,"--",pose_landmarks.part(b).y)
        print("..........................")


        with open(label, "w") as file:
            cvs_writer = writer(file)
            dist = []
            for i in range(68):
                for j in range(i+1,68):
                    dist.append(sqrt((vec[i][0]-vec[j][0])**2 + (vec[i][1]-vec[j][1])**2))
            #print(dist)
            cvs_writer.writerows(map(lambda x: [x], dist))
        dataset = pd.read_csv(label)
        sc_dist = StandardScaler()
        sc_dist = sc_dist.fit_transform(dataset)
        label = (label + "_n.csv")
        with open(label, "a", newline='') as file:
            cvs_writer = writer(file)
            cvs_writer.writerow(sc_dist)

#------------------------------------------------------------------------------------------------------


BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
image_dir = os.path.join(BASE_DIR, "images")

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('png') or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
            print(label, path)
            faceLandmarks(path,label)

dlib.hit_enter_to_continue()

          

    

abhi C:\Users\amrsa\Documents\FaceRecognition\images\abhi\0.png
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\abhi\0.png
- Face #0 found at Left: 29 Top: 47 Right: 184 Bottom: 202
..........................
abhi C:\Users\amrsa\Documents\FaceRecognition\images\abhi\1.png
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\abhi\1.png
- Face #0 found at Left: -22 Top: 12 Right: 150 Bottom: 167
..........................
abhi C:\Users\amrsa\Documents\FaceRecognition\images\abhi\2.png
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\abhi\2.png
- Face #0 found at Left: -22 Top: 12 Right: 150 Bottom: 167
..........................
abhi C:\Users\amrsa\Documents\FaceRecognition\images\abhi\3.png
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\abhi\3.png
- Face #0 found at Left: 29 Top: 47 Right: 184 Bottom: 202
..........................
abhi C:\Users\amrsa\Documents\Face

peter C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (2).jpg
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (2).jpg
- Face #0 found at Left: 76 Top: 46 Right: 165 Bottom: 136
..........................
peter C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (3).jpg
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (3).jpg
- Face #0 found at Left: 113 Top: 47 Right: 187 Bottom: 121
..........................
peter C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (4).jpg
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (4).jpg
- Face #0 found at Left: 38 Top: 53 Right: 167 Bottom: 182
..........................
peter C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (5).jpg
Found 1 faces in the image file C:\Users\amrsa\Documents\FaceRecognition\images\peter\2 (5).jpg
- Face #0 found at Left: 132 Top: 36 Right: 175 Bottom: 79
..................

# To generate the compare.csv(mean of all the distancds) file for the label.csv files

In [9]:
import numpy as np
from csv import writer, reader
import csv
import os

BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
image_dir = os.path.join(BASE_DIR, "csv")

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('csv'):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
            print(label, path)
            
            
            with open(path, "r") as file:
                csv_reader = reader(file)
                data = list(csv_reader)
                data = np.transpose(data)
                
                meanv = []
                meanv.append(label)
                for row in data:
                    sumv = 0
                    i=0
                    for x in row:
                        #print(x)
                        sumv = sumv + float(x[1:-1])
                        i+=1
                    meanv.append(sumv/i)
                    #print('------------------------------------------------')
                #print(meanv)
                with open('compare.csv', "a", newline='') as file:
                    cvs_writer = writer(file)
                    cvs_writer.writerow(meanv)
                 
                
            

abhi C:\Users\amrsa\Documents\FaceRecognition\csv\abhi\abhi_n.csv
amrendra C:\Users\amrsa\Documents\FaceRecognition\csv\amrendra\amrendra_n.csv
ankit C:\Users\amrsa\Documents\FaceRecognition\csv\ankit\ankit_n.csv
peter C:\Users\amrsa\Documents\FaceRecognition\csv\peter\peter_n.csv
ranbir C:\Users\amrsa\Documents\FaceRecognition\csv\ranbir\ranbir_n.csv


In [4]:
import numpy as np
from csv import writer, reader
import csv
import os

BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
image_dir = os.path.join(BASE_DIR, "csv")

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('csv'):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
            print(root)
            

C:\Users\amrsa\Documents\FaceRecognition\csv\abhi
C:\Users\amrsa\Documents\FaceRecognition\csv\amrendra
C:\Users\amrsa\Documents\FaceRecognition\csv\ankit
C:\Users\amrsa\Documents\FaceRecognition\csv\peter
C:\Users\amrsa\Documents\FaceRecognition\csv\ranbir


# Recognition using --video frame

In [1]:
# main block to recognise the image


import os
import cv2
from PIL import Image
import pickle
import sys
import dlib
import numpy as np
from skimage import io
from math import sqrt
from csv import writer, reader
from sklearn.preprocessing import StandardScaler
import pandas as pd
import face_recognition


#-------------------------------------------------------------------------------
def compare(data):
    min=999999999999
    with open('compare.csv', "r") as file:
        csv_reader = reader(file)
        for row in csv_reader:
            #print(row)
            sum=0
            print('comparing.................')
            for i in range (1,len(row)):
                x=float(data[i][0])
                sum = sum + (float(row[i])-x)**2
            print(row[0],"  ",sum)
            if(sum < min):
                min=sum
                name=row[0]
    if min < 30:
        print("Recognised as : Unknown")
    else:
        print("Recognised as :"+name)
#-------------------------------------------------------------------------------
def facecompare(image):
    predictor_model = "shape_predictor_68_face_landmarks.dat"

    # Create a HOG face detector using the built-in dlib class
    face_detector = dlib.get_frontal_face_detector()
    face_pose_predictor = dlib.shape_predictor(predictor_model)


    # Run the HOG face detector on the image data
    detected_faces = face_detector(image, 1)

    print("Found {} faces.".format(len(detected_faces)))

    # to get the window to show image
    #win = dlib.image_window()
    # Show the desktop window with the image
    #win.set_image(image)



    # Loop through each face we found in the image
    for i, face_rect in enumerate(detected_faces):

        # Detected faces are returned as an object with the coordinates 
        # of the top, left, right and bottom edges
        print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))

        # Draw a box around each face we found
        #win.add_overlay(face_rect)
        # Get the the face's pose
        pose_landmarks = face_pose_predictor(image, face_rect)
        # Draw the face landmarks on the screen.
        #win.add_overlay(pose_landmarks)
        
        vec = np.empty([68, 2], dtype = int)
        for b in range(68):
            vec[b][0] = pose_landmarks.part(b).x
            vec[b][1] = pose_landmarks.part(b).y
        print("..........................")


        dist = []
        for i in range(68):
            for j in range(i+1,68):
                dist.append(sqrt((vec[i][0]-vec[j][0])**2 + (vec[i][1]-vec[j][1])**2))
        #print(dist)
        dist = list(map(lambda x: [x], dist))
        dataset = (dist)
        sc_dist = StandardScaler()
        sc_dist = sc_dist.fit_transform(dataset)
        print(sc_dist)
        compare(sc_dist)
#--------------------------------------------------------------------------------
import timeit

video_cap = cv2.VideoCapture(0)
while True:
    start = timeit.default_timer()
    ret, frame = video_cap.read()
    cv2.imshow('Input', frame)
    facecompare(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    stop = timeit.default_timer()
    print("--in :",stop - start,"sec")
	
video_cap.release()
cv2.destroyAllWindows()
		
		


Found 0 faces.
--in : 5.1412195213249605 sec
Found 0 faces.
--in : 4.0463164771101265 sec
Found 0 faces.
--in : 4.001501519632052 sec
Found 0 faces.
--in : 4.343256323347754 sec
Found 1 faces.
- Face #0 found at Left: 322 Top: 219 Right: 476 Bottom: 374
..........................
[[-1.40100361]
 [-0.81751351]
 [-0.22900597]
 ..., 
 [-1.79509312]
 [-1.61472064]
 [-1.82515976]]
comparing.................
abhi    60.87330725929145
comparing.................
amrendra    203.01087867508767
comparing.................
ankit    60.221981805851506
comparing.................
peter    75.63370340889257
comparing.................
ranbir    62.12203124200794
Recognised as :ankit
--in : 4.6552530244722945 sec
Found 0 faces.
--in : 4.076957862058222 sec
Found 1 faces.
- Face #0 found at Left: 340 Top: 118 Right: 563 Bottom: 341
..........................
[[-1.43159867]
 [-0.82556964]
 [-0.241988  ]
 ..., 
 [-1.79185307]
 [-1.61252578]
 [-1.83239595]]
comparing.................
abhi    55.477714935656

Found 0 faces.
--in : 4.266287593837433 sec
Found 0 faces.


# To reconise the --image using location

In [12]:
#-------------------------------------------------------------------------------
def compare(data):
    min=999999999999
    with open('compare.csv', "r") as file:
        csv_reader = reader(file)
        for row in csv_reader:
            #print(row)
            sum=0
            print('comparing.................')
            for i in range (1,len(row)):
                x=float(data[i][0])
                sum = sum + (float(row[i])-x)**2
            print(row[0],"  ",sum)
            if(sum < min):
                min=sum
                name=row[0]
    if min < 30:
        print("Recognised as : Unknown")
    else:
        print("Recognised as :"+name)
                
#-------------------------------------------------------------------------------
def facecompare(image):
    predictor_model = "shape_predictor_68_face_landmarks.dat"

    # Create a HOG face detector using the built-in dlib class
    face_detector = dlib.get_frontal_face_detector()
    face_pose_predictor = dlib.shape_predictor(predictor_model)


    # Run the HOG face detector on the image data
    detected_faces = face_detector(image, 1)

    print("Found {} faces.".format(len(detected_faces)))

    # to get the window to show image
    win = dlib.image_window()
    # Show the desktop window with the image
    win.set_image(image)



    # Loop through each face we found in the image
    for i, face_rect in enumerate(detected_faces):

        # Detected faces are returned as an object with the coordinates 
        # of the top, left, right and bottom edges
        print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))

        # Draw a box around each face we found
        win.add_overlay(face_rect)
        # Get the the face's pose
        pose_landmarks = face_pose_predictor(image, face_rect)
        # Draw the face landmarks on the screen.
        win.add_overlay(pose_landmarks)
        
        vec = np.empty([68, 2], dtype = int)
        for b in range(68):
            vec[b][0] = pose_landmarks.part(b).x
            vec[b][1] = pose_landmarks.part(b).y
        print("..........................")


        dist = []
        for i in range(68):
            for j in range(i+1,68):
                dist.append(sqrt((vec[i][0]-vec[j][0])**2 + (vec[i][1]-vec[j][1])**2))
        #print(dist)
        dist = list(map(lambda x: [x], dist))
        dataset = (dist)
        sc_dist = StandardScaler()
        sc_dist = sc_dist.fit_transform(dataset)
        print(sc_dist)
        compare(sc_dist)
#--------------------------------------------------------------------------------


image = cv2.imread('ankit.jpg')
facecompare(image)
	
#video_cap.release()
cv2.destroyAllWindows()
		
		


Found 1 faces.
- Face #0 found at Left: 196 Top: 82 Right: 325 Bottom: 211
..........................
[[-1.46293709]
 [-0.91973248]
 [-0.37201798]
 ..., 
 [-1.79875633]
 [-1.56440397]
 [-1.79875633]]
comparing.................
abhi    103.50044788168465
comparing.................
amrendra    79.05462179242329
comparing.................
ankit    0.8988940384572378
comparing.................
peter    28.231394168709024
comparing.................
ranbir    29.887267818313767
Recognised as :ankit
